In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('dataset/heart.csv')
df.head()

In [ ]:
df["result"]=df[" result"].replace({'negative':0,'positive':1})
df.head(1)

In [ ]:
df.info()

In [ ]:
for sutun_adi in df.columns:
    print("{} sütunundaki benzersiz değerler : {}".format(sutun_adi, df[sutun_adi].unique()))

In [ ]:
df["ca"]= df["ca"].replace("<null>", 'Nan' )
df["thal"]= df["thal"].replace("<null>", 'Nan' )

In [ ]:
df['ca'] = df['ca'].astype(int)
df['thal'] = df['thal'].astype(int)

In [ ]:
df.fillna(df.mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
sns.countplot(df.result)
print("Kalp Krizi Oranı : %{:.2f}".format(sum(df.result)/len(df.result)*100))
print("Risk Yok      : %{:.2f}".format((len(df.result)-sum(df.result))/len(df.result)*100))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def model_olustur(X, y):
    X_eğitim, X_test, y_eğitim, y_test =  train_test_split(X, y, test_size=0.20, random_state=111, stratify = y)
    logreg_model = LogisticRegression()
    logreg_model.fit(X_eğitim, y_eğitim)

    tahmin_eğitim = logreg_model.predict(X_eğitim)
    tahmin_test = logreg_model.predict(X_test)
    hata_matrisi_eğitim = confusion_matrix(y_eğitim, tahmin_eğitim)
    hata_matrisi_test = confusion_matrix(y_test, tahmin_test)
    print("Modelin doğruluk değeri : ",  logreg_model.score(X_test, y_test))
    print("Eğitim veri kümesi")
    print(classification_report(y_eğitim,tahmin_eğitim) )
    print("Test veri kümesi")
    print(classification_report(y_test,tahmin_test) )
    return  None

In [ ]:
df=df.drop('ca', axis=1)
df=df.drop('thal', axis=1)

In [ ]:
X = df.drop('result', axis=1)
y = df['result']
model_olustur(X,y)

In [ ]:
Kalp_krizi = df[df.result == 1]
Risk_yok = df[df.result == 0]

Kalp_krizi_artırılmış = resample(Kalp_krizi,
                                     replace = True,
                                     n_samples = len(Risk_yok),
                                     random_state = 111)

artırılmıs_df = pd.concat([ Kalp_krizi_artırılmış,Risk_yok])
artırılmıs_df.result.value_counts()


In [ ]:
X = artırılmıs_df.drop('result', axis=1)
y = artırılmıs_df['result']
model_olustur(X,y)

In [ ]:
Kalp_krizi = df[df.result == 1]
Risk_yok = df[df.result == 0]

Kalp_krizi_azaltılmış = resample(Risk_yok,
                                     replace = True,
                                     n_samples = len(Kalp_krizi),
                                     random_state = 111)

azaltılmış_df = pd.concat([Kalp_krizi, Kalp_krizi_azaltılmış])
azaltılmış_df.result.value_counts()

In [ ]:

X = azaltılmış_df.drop('result', axis=1)
y = azaltılmış_df['result']
model_olustur(X,y)

In [ ]:
y = df.result
X = df.drop('result', axis=1)

sm = SMOTE(random_state=27)
X_smote, y_smote = sm.fit_sample(X, y)

In [ ]:
model_olustur(X_smote, y_smote)

In [ ]:
y = df.result
X = df.drop('result', axis=1)

ad = ADASYN()
X_adasyn, y_adasyn = ad.fit_sample(X, y)

In [ ]:
model_olustur(X_adasyn, y_adasyn)